参考
- <http://blog.yhat.com/posts/logistic-regression-python-rodeo.html>
- <http://stats.stackexchange.com/questions/203740/logistic-regression-scikit-learn-vs-statsmodels>

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [32]:
auto = pd.read_csv('auto.csv',encoding = 'gb2312')
auto.head()

EngSize  Age Gender Marital  exp Owner  vAge Garage AntiTFD import  Loss
0      2.0   56      男      已婚   20    公司    10      有   有防盗装置     进口   0.0
1      1.8   41      男      已婚   20    公司     9      有   无防盗装置     国产   0.0
2      2.0   44      男      未婚   20    公司     8      有   有防盗装置     国产   0.0
3      1.6   56      男      已婚   20    公司     7      有   有防盗装置     国产   0.0
4      1.8   45      男      已婚   20    公司     7      无   无防盗装置     国产   0.0

In [33]:
# ?np.genfromtxt

In [34]:
# 变量取值的转换

auto['LossClass'] = auto['Loss'].map(lambda x: 1 if x > 0 else 0)

In [35]:
auto.head()

EngSize  Age Gender Marital  exp Owner  vAge Garage AntiTFD import  Loss  \
0      2.0   56      男      已婚   20    公司    10      有   有防盗装置     进口   0.0   
1      1.8   41      男      已婚   20    公司     9      有   无防盗装置     国产   0.0   
2      2.0   44      男      未婚   20    公司     8      有   有防盗装置     国产   0.0   
3      1.6   56      男      已婚   20    公司     7      有   有防盗装置     国产   0.0   
4      1.8   45      男      已婚   20    公司     7      无   无防盗装置     国产   0.0   

   LossClass  
0          0  
1          0  
2          0  
3          0  
4          0

In [36]:
# 二分类变量的转换
auto['Gender'] = auto['Gender'].map(lambda x: 1 if x == u'女'else 0)

auto['Marital'] = auto['Marital'].map(lambda x: 1 if x == u'已婚' else 0)

auto['Garage'] = auto['Garage'].map(lambda x: 1 if x == u'有'else 0)

auto['AntiTFD'] = auto['AntiTFD'].map(lambda x: 1 if x == u'有防盗装置'else 0)

auto['import'] = auto['import'].map(lambda x: 1 if x == u'进口'else 0)


# 变量取值的转换

auto['vAgeNew'] = auto['vAge'].map(lambda x: 1 if x == 1 else 0)

auto['EngType'] = auto['EngSize'].map(lambda x: 1 if x >1.6 else 0)

auto['Owner'] = auto['Owner'].replace({u'公司':1, u'私人':2, u'政府':3})

auto['Intercept'] = 1.0

# 另一种写法
# auto['Owner'] = auto['Owner'].replace([u'公司', u'私人', u'政府'],[1,2,3])

auto.head()

EngSize  Age  Gender  Marital  exp  Owner  vAge  Garage  AntiTFD  import  \
0      2.0   56       0        1   20      1    10       1        1       1   
1      1.8   41       0        1   20      1     9       1        0       0   
2      2.0   44       0        0   20      1     8       1        1       0   
3      1.6   56       0        1   20      1     7       1        1       0   
4      1.8   45       0        1   20      1     7       0        0       0   

   Loss  LossClass  vAgeNew  EngType  Intercept  
0   0.0          0        0        1        1.0  
1   0.0          0        0        1        1.0  
2   0.0          0        0        1        1.0  
3   0.0          0        0        0        1.0  
4   0.0          0        0        1        1.0

In [37]:
owner_ranks = pd.get_dummies(auto['Owner'], prefix='Owner')
owner_ranks.head()

Owner_1  Owner_2  Owner_3
0        1        0        0
1        1        0        0
2        1        0        0
3        1        0        0
4        1        0        0

In [38]:
auto.columns

Index([  u'EngSize',       u'Age',    u'Gender',   u'Marital',       u'exp',
           u'Owner',      u'vAge',    u'Garage',   u'AntiTFD',    u'import',
            u'Loss', u'LossClass',   u'vAgeNew',   u'EngType', u'Intercept'],
      dtype='object')

In [39]:
cols_to_keep = ['LossClass','Intercept', 'Age', 'Gender','Marital','exp',\
                'vAgeNew','Garage','AntiTFD','import','EngType' ]
                 
auto = auto[cols_to_keep].join(owner_ranks.ix[:, 'Owner_2':])

auto.head()

/usr/local/src/anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  This is separate from the ipykernel package so we can avoid doing imports until


LossClass  Intercept  Age  Gender  Marital  exp  vAgeNew  Garage  AntiTFD  \
0          0        1.0   56       0        1   20        0       1        1   
1          0        1.0   41       0        1   20        0       1        0   
2          0        1.0   44       0        0   20        0       1        1   
3          0        1.0   56       0        1   20        0       1        1   
4          0        1.0   45       0        1   20        0       0        0   

   import  EngType  Owner_2  Owner_3  
0       1        1        0        0  
1       0        1        0        0  
2       0        1        0        0  
3       0        0        0        0  
4       0        1        0        0

In [40]:
y = auto['LossClass']
X = auto.iloc[:,1:]

In [41]:
X.head()

Intercept  Age  Gender  Marital  exp  vAgeNew  Garage  AntiTFD  import  \
0        1.0   56       0        1   20        0       1        1       1   
1        1.0   41       0        1   20        0       1        0       0   
2        1.0   44       0        0   20        0       1        1       0   
3        1.0   56       0        1   20        0       1        1       0   
4        1.0   45       0        1   20        0       0        0       0   

   EngType  Owner_2  Owner_3  
0        1        0        0  
1        1        0        0  
2        1        0        0  
3        0        0        0  
4        1        0        0

# 逻辑回归

In [42]:
from sklearn.linear_model import LogisticRegression  
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report   
from sklearn import metrics  


X_train,X_test,y_train,y_test = train_test_split(
X,y,test_size=0.25,random_state=42)
  
# 使用类，参数全是默认的  
# classifier = LogisticRegression(fit_intercept = False, C = 1e9,class_weight="balanced")  
classifier = LogisticRegression(fit_intercept = False, C = 1e9,class_weight="balanced")  
model = classifier.fit(X_train, y_train)               # 训练数据来学习  
  

In [43]:
?train_test_split

In [44]:
y_pre = model.predict(X_test)
print(classification_report(y_test,y_pre))  
metrics.roc_auc_score(y_test,y_pre) #预测Y值得分

             precision    recall  f1-score   support

          0       0.77      0.54      0.64       751
          1       0.35      0.62      0.45       308

avg / total       0.65      0.56      0.58      1059



0.57808203780241074

TP: 将正类预测为正类数  

FN: 将正类预测为负类数  

FP: 将负类预测为正类数  

TN: 将负类预测为负类数  


准确率(accuracy) = 预测对的/所有 = (TP+TN)/(TP+FN+FP+TN)

精确率(precision) = TP/(TP+FP) 

召回率(recall) = TP/(TP+FN) 


In [45]:
pd.DataFrame(zip(X.columns, np.transpose(model.coef_)))

0                    1
0   Intercept    [-0.232425284095]
1         Age  [-0.00304250842626]
2      Gender    [0.0118034813204]
3     Marital     [0.157649033635]
4         exp   [-0.0253148701257]
5     vAgeNew     [0.466069802059]
6      Garage     [0.140933420115]
7     AntiTFD    [0.0868162504313]
8      import      [0.11997442402]
9     EngType    [-0.332963328226]
10    Owner_2     [0.204654769193]
11    Owner_3    [-0.510590625327]

# 梯度下降分类

In [46]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss="hinge", penalty="l2",class_weight="balanced")
SGD = clf.fit(X_train, y_train)

In [47]:
y_pre = SGD.predict(X_test)
print(classification_report(y_test,y_pre))  
metrics.roc_auc_score(y_test,y_pre) #预测Y值得分

             precision    recall  f1-score   support

          0       0.80      0.32      0.46       751
          1       0.33      0.81      0.46       308

avg / total       0.66      0.46      0.46      1059



0.56238435332975945

# 决策树

In [48]:
from sklearn import tree

In [49]:
DT = tree.DecisionTreeClassifier(criterion ='entropy',class_weight="balanced")         #criterion default=”gini”
DT = DT.fit(X_train, y_train)

In [50]:
y_pre = DT.predict(X_test)
print(classification_report(y_test,y_pre))  
metrics.roc_auc_score(y_test,y_pre) #预测Y值得分

             precision    recall  f1-score   support

          0       0.73      0.71      0.72       751
          1       0.33      0.36      0.35       308

avg / total       0.61      0.61      0.61      1059



0.53276583602815297

# 随机森林

In [51]:
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

In [52]:
clf = DecisionTreeClassifier(max_depth=3, min_samples_split=10,random_state=0,class_weight="balanced")
clf = clf.fit(X_train, y_train) 
y_pre = clf.predict(X_test)
print(classification_report(y_test,y_pre))  
metrics.roc_auc_score(y_test,y_pre) #预测Y值得分

             precision    recall  f1-score   support

          0       0.76      0.56      0.64       751
          1       0.34      0.56      0.43       308

avg / total       0.64      0.56      0.58      1059



0.56017950092517343

In [53]:
clf = RandomForestClassifier(n_estimators=250, max_depth=5 ,min_samples_split=10, random_state=0,class_weight="balanced")
clf = clf.fit(X_train, y_train) 
y_pre = clf.predict(X_test)
print(classification_report(y_test,y_pre))  
metrics.roc_auc_score(y_test,y_pre) #预测Y值得分

             precision    recall  f1-score   support

          0       0.77      0.56      0.64       751
          1       0.35      0.58      0.44       308

avg / total       0.64      0.56      0.58      1059



0.57050339806664696

# 用随机森林做特征选择（特征重要性排序）

In [54]:
importances = clf.feature_importances_

In [55]:
std = np.std([tree.feature_importances_ for tree in clf.estimators_],
             axis=0)

In [56]:
indices = np.argsort(importances)[::-1]     

In [63]:
# Print the feature ranking
print("Feature ranking:")
for f in range(X.shape[1]):
    print("%d. feature %d (%f): %s" % (f + 1, indices[f], importances[indices[f]] , X.columns[f] ))

Feature ranking:
1. feature 5 (0.221236): Intercept
2. feature 4 (0.201298): Age
3. feature 1 (0.160727): Gender
4. feature 10 (0.115707): Marital
5. feature 9 (0.079040): exp
6. feature 11 (0.077910): vAgeNew
7. feature 6 (0.054507): Garage
8. feature 8 (0.024897): AntiTFD
9. feature 7 (0.023193): import
10. feature 2 (0.022324): EngType
11. feature 3 (0.019161): Owner_2
12. feature 0 (0.000000): Owner_3


In [64]:

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()

In [65]:
X.columns

Index([u'Intercept',       u'Age',    u'Gender',   u'Marital',       u'exp',
         u'vAgeNew',    u'Garage',   u'AntiTFD',    u'import',   u'EngType',
         u'Owner_2',   u'Owner_3'],
      dtype='object')

In [66]:
clf = ExtraTreesClassifier(n_estimators=250, max_depth=5,min_samples_split=2, random_state=0,class_weight="balanced")
clf = clf.fit(X_train, y_train) 
y_pre = clf.predict(X_test)
print(classification_report(y_test,y_pre))  
metrics.roc_auc_score(y_test,y_pre) #预测Y值得分

             precision    recall  f1-score   support

          0       0.76      0.56      0.64       751
          1       0.35      0.57      0.43       308

avg / total       0.64      0.56      0.58      1059



0.56534144949591014

参考：http://scikit-learn.org/stable/modules/ensemble.html#random-forests

# 支持向量分类

In [ ]:
from sklearn import svm
clf = svm.SVC(kernel = 'linear',decision_function_shape='ovo',class_weight="balanced")
clf = clf.fit(X_train, y_train) 
y_pre = clf.predict(X_test)
print(classification_report(y_test,y_pre))  
metrics.roc_auc_score(y_test,y_pre) #预测Y值得分

# K近邻分类


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=3)
clf = clf.fit(X_train, y_train) 
y_pre = clf.predict(X_test)
print(classification_report(y_test,y_pre))  
metrics.roc_auc_score(y_test,y_pre) #预测Y值得分

# 核近似


In [ ]:
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier

rbf_feature = RBFSampler(gamma=1, random_state=1)
X_features = rbf_feature.fit_transform(X_train)
aa  =  rbf_feature.fit_transform(X_test)

In [ ]:
clf = SGDClassifier()   
clf = clf.fit(X_features, y_train)

In [ ]:
y_pre = clf.predict(aa)
print(classification_report(y_test,y_pre))  
metrics.roc_auc_score(y_test,y_pre) #预测Y值得分

# 朴素贝叶斯（多用于文本分类）

# ADBOOST

In [ ]:
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=5, random_state=0)
clf = clf.fit(X_train, y_train)
y_pre = clf.predict(X_test)
print(classification_report(y_test,y_pre))  
metrics.roc_auc_score(y_test,y_pre) #预测Y值得分

In [ ]:
?GradientBoostingClassifier